In [1]:
from pipeline.pipeline import Pipeline
from music21 import converter, midi, key, instrument, meter, interval, note, pitch, stream, tempo
import os
import pandas as pd
import mido
import glob

import pretty_midi

import mido
import pandas as pd

from utils.utils import open_midi, get_file_name, extract_tempo, extract_key, to_snake_case

In [2]:
midi_folder = './test_data/processed'
output_folder = './test_data/cropped'
num_bars = 8

p_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/program_change.csv')
com_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/commu_meta.csv', index_col=0)

In [3]:
def get_tempo(pm):
    tempo_estimated = pm.estimate_tempo()
    _, tempo = pm.get_tempo_changes()
    return tempo.tolist()

def find_near_bpm(bpm_list, bpm_value):
    answer = 0
    minValue = min(bpm_list, key=lambda x:abs(x-bpm_value))
    answer = minValue
    return answer

In [4]:
bpm_list = com_df['bpm'].unique().tolist()
bpm_list.sort()

In [5]:
bpm_list = [35,45,50,60,65,70,75,80,85,90,95,100,105,110,120,125,130,135,140,145,150,155]

In [6]:
midi_path = glob.glob(f'{midi_folder}/*.mid')

In [7]:
midi_path

['./test_data/processed/Abyss.mid',
 './test_data/processed/CoronationStreet.mid',
 './test_data/processed/FabulCastle.mid',
 './test_data/processed/FigaroCastle.mid',
 './test_data/processed/FinalBattle.mid',
 './test_data/processed/1492ConquestofParadise.mid',
 './test_data/processed/10.mid',
 './test_data/processed/Hyperdimensionneptuniatheanimationopeningdimensiontripper.mid',
 './test_data/processed/FinalFantasyIX-MelodiesOfLife.mid',
 './test_data/processed/CharlieAndLola.mid',
 './test_data/processed/Dr.mid',
 './test_data/processed/FinalBattle_1.mid',
 './test_data/processed/HouseM.mid']

In [8]:
mid = mido.MidiFile(midi_path[8])

In [9]:
score1 = open_midi(midi_path[8])
score2 = open_midi(midi_path[8])

In [10]:
beats_per_bar = score1.recurse().getElementsByClass(meter.TimeSignature)[0].numerator
num_measures = int(score1.duration.quarterLength / beats_per_bar)

In [11]:
extracted_tempo = extract_tempo(score1)
target_bpm = find_near_bpm(bpm_list, extracted_tempo)

# for part in score1.parts:

#     tempo_measures = []

#     for measure in part.measures(0, None):
#         if measure.recurse().getElementsByClass(tempo.MetronomeMark):
#             tempo_measures.append(measure.measureNumber)

#     for idx, value in enumerate(tempo_measures):
#         if idx < len(tempo_measures) - 1:
#             for i in range(6):
#                 try:
#                     part.measures(value, tempo_measures[idx+1]).recurse().getElementsByClass(tempo.MetronomeMark)[i].number = target_bpm
#                 except:
#                     pass

#         elif idx == len(tempo_measures) - 1:

#             for i in range(6):
#                 try:
#                     part.measures(value, tempo_measures[idx]).recurse().getElementsByClass(tempo.MetronomeMark)[i].number = target_bpm
#                 except:
#                     pass


In [12]:
numbers = []
for part in score1.parts:

    for measure in part.measures(0, None):

        if measure.recurse().getElementsByClass(tempo.MetronomeMark):
            for i in range(6):
                try:
                    measure.recurse().getElementsByClass(tempo.MetronomeMark)[i].number = target_bpm
                except:
                    pass
        

In [13]:
for i in range(5):
    try: 
        print(score1.parts[1].measures(3, 4).recurse().getElementsByClass(tempo.MetronomeMark)[i])
    except:
        pass

<music21.tempo.MetronomeMark adagietto Quarter=70>
<music21.tempo.MetronomeMark andante Quarter=70>
<music21.tempo.MetronomeMark adagietto Quarter=70>


In [20]:
for i in range(len(score1.parts)):
    for x in score1.parts[i].recurse().getElementsByClass(tempo.MetronomeMark):
        x.number = target_bpm
